In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import timedelta


In [ ]:
def get_data(url):
    # need to set a user agent to avoid 403 Forbidden error
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        # Parse the HTML content of the page with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the first table in the page (you can refine the selection)
        table = soup.find('table')

        # Check if a table was found
        if table:
            # Extract rows from the table
            rows = table.find_all('tr')

            # Prepare a list to hold the data
            data = []

            # Iterate over the rows
            for row in rows[1:]:
                # Extract cells in the row
                cells = row.find_all(['td', 'th'])  # Include both <td> and <th> cells

                # Extract text and links from each cell
                cell_data = []
                for cell in cells:

                    # Check if the cell contains a <div> tag
                    if cell.find('div'):
                        cell = cell.find('div')

                    # Extract text from the cell
                    text = cell.get_text(strip=True)
                    
                    # Extract link if the cell contains an <a> tag
                    link = cell.find('a')['href'] if cell.find('a') else None

                    if link and link.startswith('https://www.parkrun.co.za/albertsfarm/parkrunner/'):
                        cell_data.append(link)
                        cell_data.append(link.split('/')[-1])
                    
                    cell_data.append(text)
                
                # Add the row data to the main data list
                data.append(cell_data)  
            
            return data
        else:
            print("No table found on the page.")
            return None
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
    return None

In [ ]:
alberts_1 = "https://www.parkrun.co.za/albertsfarm/results/1/"
alberts_1_data = get_data(alberts_1)
alberts_1_data

In [ ]:
alberts_10 = "https://www.parkrun.co.za/albertsfarm/results/438/"
alberts_10_data = get_data(alberts_10)
alberts_10_data

In [ ]:
def convert_time_string(time_str):
    # Split the time string by colon
    time_parts = time_str.split(':')
    
    # Determine if it's in MM:SS format or HH:MM:SS format
    if len(time_parts) == 2:  # MM:SS
        minutes, seconds = map(int, time_parts)
        return timedelta(minutes=minutes, seconds=seconds)
    elif len(time_parts) == 3:  # HH:MM:SS
        hours, minutes, seconds = map(int, time_parts)
        return timedelta(hours=hours, minutes=minutes, seconds=seconds)
    else:
        return None

In [ ]:
df1 = pd.DataFrame(alberts_1_data)
df1.columns = ['pos', 'link', 'barcode', 'name', 'gender', 'age', 'club', 'time']
df1['barcode'] = df1['barcode'].apply(lambda x: "A" + x)
# remove rows with name is None
df1 = df1.dropna(subset=['name'])
df1 = df1.dropna(subset=['time'])
df1['time_delta'] = df1['time'].apply(convert_time_string)
df1

In [ ]:
df10 = pd.DataFrame(alberts_10_data)
df10.columns = ['pos', 'link', 'barcode', 'name', 'gender', 'age', 'club', 'time']
df10['barcode'] = df10['barcode'].apply(lambda x: "A" + x)
# remove rows with name is None
df10 = df10.dropna(subset=['name'])
df10 = df10.dropna(subset=['time'])
df10['time_delta'] = df10['time'].apply(convert_time_string)
df10

In [ ]:
# check who ran both
both = pd.merge(df1, df10, on='barcode', how='inner')
both

In [ ]:
# Function to convert seconds to MM:SS format
def format_seconds_to_mm_ss(seconds):
    # Ensure the seconds are positive for formatting
    abs_seconds = abs(int(seconds))
    
    # Calculate minutes and remaining seconds
    minutes, sec = divmod(abs_seconds, 60)
    
    # Format as MM:SS
    formatted_time = f"{minutes:02}:{sec:02}"
    
    # Add a negative sign if the original seconds were negative
    return f"-{formatted_time}" if seconds < 0 else formatted_time

In [ ]:
# show the difference in time_delta column
both['time_delta_diff'] = (both['time_delta_y'] - both['time_delta_x']).apply(lambda x: x.total_seconds())

both['formatted_time_diff'] = both['time_delta_diff'].apply(format_seconds_to_mm_ss)

both


In [ ]:
both_simple = both[['name_x', 'barcode', 'age_y', 'time_x', 'pos_x', 'time_y', 'pos_y', 'formatted_time_diff']]
both_simple.columns = ['Name', 'Barcode', 'Age Group', 'Time 1', 'Pos 1', 'Time 10', 'Pos 10', 'Time Diff']
both_simple